# Intuition for Publication Bias in Python Notebook
This notebook demonstrates the concept of publication bias interactively using Python, NumPy, Plotly, and ipywidgets.

In [ ]:

# Import necessary libraries
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact


In [ ]:

# Function to create the bias plot
def plot_bias(s2):
    s = 1 - s2

    dark_blue = "#1f77b4"
    dark_orange = "#ff7f0e"

    x1 = np.linspace(-3, 3, 500)
    y1 = np.exp(-0.5 * x1**2) / np.sqrt(2 * np.pi)

    x2 = np.linspace(-3, 3, 500)
    y2 = np.exp(-0.5 * x2**2)
    y2[x2 < 2] *= s
    y2 /= np.trapz(y2, x2)

    avg_z1 = 0
    avg_z2 = np.trapz(x2 * y2, x2)

    bar_x = ["p<0.05, neg", "NS, neg", "NS, pos", "p<0.05, pos"]
    bar_y1 = [0.025, 0.475, 0.475, 0.025]
    bar_y2 = bar_y1[:]
    bar_y2[:3] = [val * s for val in bar_y1[:3]]
    bar_y2 = [val / sum(bar_y2) for val in bar_y2]

    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=False, vertical_spacing=0.1,
        subplot_titles=("Distribution of Real Effects",
                        "Distribution of Published Papers",
                        "Significance of Experiments",
                        "Significance of Published Papers"))

    fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', line=dict(color=dark_blue), name="Real Effects"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', line=dict(color=dark_orange), name="Published Papers"), row=2, col=1)
    fig.add_trace(go.Bar(x=bar_x, y=bar_y1, marker_color=dark_blue, name="Significance of Experiments"), row=3, col=1)
    fig.add_trace(go.Bar(x=bar_x, y=bar_y2, marker_color=dark_orange, name="Significance of Published Papers"), row=4, col=1)

    fig.update_layout(
        height=800,
        margin=dict(t=100, b=50, l=50, r=50),
        title_x=0.5,
        showlegend=False
    )

    fig.update_xaxes(title_text="Z-score (mean/std)", row=1, col=1)
    fig.update_xaxes(title_text="Z-score (mean/std)", row=2, col=1)

    return fig


In [ ]:

# Interactive plot function using ipywidgets
@interact(s2=widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='Unpublished %'))
def update_plot(s2):
    fig = plot_bias(s2)
    fig.show()
